In [68]:
from tqdm.auto import tqdm
import time
import subprocess
import random

In [69]:
wa = False

class Test():
    def __init__(self):
        self.generate_test()
        
    def generate_test(self): 
        with open("test.in", 'w') as test:
            self.n = random.randint(2, 1_000)
            pts = set()
            
            test.write(f"{self.n}\n")
            
            for _ in range(self.n):
                a, b = random.randint(int(-1e7), int(1e7)) , random.randint(int(-1e7), int(1e7))
                while (a, b) in pts:
                    a, b = random.randint(int(-1e7), int(1e7)) , random.randint(int(-1e7), int(1e7))
                
                pts.add((a, b)) 
                test.write(f"{a} {b}\n")
        
    def check_solutions(self, a_out, b_out):
        global wa
        # a: brut, b: better_solution
        with open(a_out, 'r') as a_out, open(b_out, 'r') as b_out:
            self.test_data_brut = [list(map(int, a_out.readline().strip().split())),
                           list(map(int, a_out.readline().strip().split()))]
    
            self.test_data_chad = [list(map(int, b_out.readline().strip().split())),
                           list(map(int, b_out.readline().strip().split()))] 
                
                
            self.better_check()
    
    def better_check(self):
        global wa
        
        def dist(a_x, a_y, b_x, b_y):
            return (a_x - b_x)**2 + (a_y - b_y)**2
        
        d_brut = dist(self.test_data_brut[0][0], self.test_data_brut[0][1], 
                      self.test_data_brut[1][0], self.test_data_brut[1][1])
        
        d_chad = dist(self.test_data_chad[0][0], self.test_data_chad[0][1],
                      self.test_data_chad[1][0], self.test_data_chad[1][1])
         
        if d_brut != d_chad:
            print("WA")
            wa = True
            return

In [70]:
# convention a: brut, b: better_solutoin
class Referee():
    def __init__(self, program_a: str, program_b: str, max_time):
        # program a = sth.cpp (give full name with .cpp)
        
        self.program_a = program_a
        self.program_b = program_b
        self.max_time = max_time
        
        # compile programs 
        self.exe_a = self.compile(program_a)
        self.exe_b = self.compile(program_b)
        
    def compile(self, program: str) -> str:
        exe_name = program.split('.')[0]
        compile_command = ['g++', program, '-o', exe_name]
        subprocess.run(compile_command, check=True)
        return exe_name
    
    def do_tests(self): 
        self.times = []
        for _ in tqdm(range(200000)): 
            # create test 
            self.test = Test()
            
            # test program_a
            start_time_a = time.time()
            with open("test.in", "r") as test_input, open(f"{self.exe_a}.out", "w") as output_a:
                try:
                    subprocess.run(f"./{self.exe_a}", stdin=test_input, stdout=output_a, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_a} exceeded time limit")
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_a} ended with an error")
                    exit(2)
            end_time_a = time.time()
            
            # test program_b
            start_time_b = time.time()
            with open("test.in", "r") as test_input, open(f"{self.exe_b}.out", "w") as output_b:
                try:
                    subprocess.run(f"./{self.exe_b}", stdin=test_input, stdout=output_b, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_b} exceeded time limit") 
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_b} ended with an error")
                    exit(2)
            end_time_b = time.time()
            
            
            # check solution
            global wa
            wa = False
            self.test.check_solutions(f"{self.exe_a}.out", f"{self.exe_b}.out")
            if wa == True:
               break 
            
            # solutions are ok
            self.times.append([end_time_a - start_time_a, end_time_b - start_time_b])
        
        else:
            print("everything ok 🚀") 

In [ ]:
referee = Referee('brut_solution.cpp', 'gigachad_solution.cpp', 2)
referee.do_tests()

 30%|███       | 60913/200000 [04:28<10:24, 222.72it/s]

In [ ]:
import pandas as pd
df = pd.DataFrame(referee.times)
df.columns = ['brut', 'chad']
df = df.sort_values(by='chad', ascending=False)
df

,brut,chad
186,1.016922,0.666397
130,0.966732,0.666326
113,0.966568,0.666054
6,0.966677,0.666026
100,0.966830,0.666015
...,...,...
32,0.015829,0.015677
19,0.031882,0.015664
44,0.015912,0.007755
173,0.015761,0.007577


In [ ]:
df.mean()
df.sort_values(by='brut', ascending=False)

,brut,chad
3,1.017123,0.665889
163,1.016966,0.665409
186,1.016922,0.666397
85,1.016307,0.665233
33,1.016117,0.615006
...,...,...
44,0.015912,0.007755
32,0.015829,0.015677
173,0.015761,0.007577
101,0.015753,0.015856
